In [ ]:
!pip install pillow
!pip install numpy
!pip install -U scikit-learn
!python -m pip install "tensorflow<2.11"

# Libraries

In [14]:
from PIL import Image
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

import cv2
import torch
import torch.nn as nn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


from Library import CustomDataset

torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 SUPER'

# Conversion in dataset

In [15]:
path='../Dataset/indoors/'
create_df_ipcv_diode(path)

# Split dataset

In [16]:
csv_path="../Csv/path_images.csv"
custom_dataset = CustomDataset(csv_path, test_size=0.3, random_state=42)
train_loader = torch.utils.data.DataLoader(custom_dataset, batch_size=16, shuffle=True)

In [17]:
import torch.nn as nn

class ImageConverter(nn.Module):
    def __init__(self):
        super(ImageConverter, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 1, kernel_size=1, stride=1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [18]:
import torch.nn as nn

class ImageConverter(nn.Module):
    def __init__(self):
        super(ImageConverter, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 1, kernel_size=1, stride=1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [19]:
model = ImageConverter()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5


In [32]:
# Imposta il dispositivo (GPU o CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Addestra il modello
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)# Trasferisci i dati al dispositivo
        optimizer.zero_grad()# Resetta i gradienti
        outputs = model(images)# Propaga in avanti
        loss = criterion(outputs, images)# Calcola la loss
        loss.backward()        # Propaga all'indietro e ottieni i gradienti
        optimizer.step()# Aggiorna i pesi
        train_loss += loss.item() * images.size(0)# Aggiorna la loss

    train_loss = train_loss / len(train_loader.dataset) # Calcola la loss media per epoca


TypeError: cannot unpack non-iterable int object

In [46]:
for i, batch in enumerate(train_loader):
    images, labels = batch

IndexError: list index out of range

In [45]:
train_loader = torch.utils.data.DataLoader(custom_dataset, batch_size=16, shuffle=True, drop_last=True)
